<h2 style="text-align:center;font-size:200%;;">工业事故原因分析</h2>
<h4 style="text-align:center;font-size:80%;;">小组成员：陈佳禾，曹栋承，邵嘉豪</h4>

# 1. 环境准备

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import tokenize,stem
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
import graphviz
import lightgbm as lgb
import nltk
from nltk.util import ngrams
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from wordcloud import WordCloud, STOPWORDS
import shap

shap.initjs()
nltk.download("maxent_treebank_pos_tagger")
nltk.download("maxent_ne_chunker")
nltk.download("punkt")
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('vader_lexicon')


# 2. 导入数据集

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv("E:\我浙\大三上\课业\大数据\大作业\Big-Data-Analysis-and-Application_Project-main (1)\Big-Data-Analysis-and-Application_Project-main\Baseline\Data\IHMStefanini_industrial_safety_and_health_database_with_accidents_description.csv")
df.head(3)

In [ ]:
df.drop("Unnamed: 0", axis=1, inplace=True)
df.rename(columns={'Data':'Date', 'Countries':'Country', 'Genre':'Gender', 'Employee or Third Party':'Employee type'}, inplace=True)
df.head(3)

# 3. 数据预处理

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].apply(lambda x : x.year)
df['Month'] = df['Date'].apply(lambda x : x.month)
df['Day'] = df['Date'].apply(lambda x : x.day)
df['Weekday'] = df['Date'].apply(lambda x : x.day_name())
df['WeekofYear'] = df['Date'].apply(lambda x : x.weekofyear)
df.head(3)

In [ ]:
def month2seasons(x):
    if x in [9, 10, 11]:
        season = 'Spring'
    elif x in [12, 1, 2]:
        season = 'Summer'
    elif x in [3, 4, 5]:
        season = 'Autumn'
    elif x in [6, 7, 8]:
        season = 'Winter'
    return season

In [ ]:
df['Season'] = df['Month'].apply(month2seasons)
df.head(3)

In [ ]:
STOPWORDS.update(["cm", "kg", "mr", "wa" ,"nv", "ore", "da", "pm", "am", "cx"])
print(STOPWORDS)

In [ ]:
def nlp_preprocesser(row):
    sentence = row.Description
    #convert all characters to lowercase
    lowered = sentence.lower()
    tok = tokenize.word_tokenize(lowered)

    #lemmatizing & stemming
    lemmatizer = stem.WordNetLemmatizer()
    lem = [lemmatizer.lemmatize(i) for i in tok if i not in STOPWORDS]
    stemmer = stem.PorterStemmer()
    stems = [stemmer.stem(i) for i in lem if i not in STOPWORDS]

    #remove non-alphabetical characters like '(', '.' or '!'
    alphas = [i for i in stems if i.isalpha() and (i not in STOPWORDS)]
    return " ".join(alphas)

In [ ]:
df['Description_processed'] = df.apply(nlp_preprocesser, axis=1)
df.head(3)

In [ ]:
def sentiment2score(text):
    analyzer = SentimentIntensityAnalyzer()
    sent_score = analyzer.polarity_scores(text)["compound"]
    return float(sent_score)

In [ ]:
df['Description_sentiment_score'] = df['Description'].apply(lambda x: sentiment2score(x))
df.head(3)

# 4. 探索性数据分析 & NLP分析

In [ ]:
## 有修改！！
country_cnt = np.round(df['Country'].value_counts(normalize=True) * 100) # 百分数取整\
#country_cnt = df['Country'].value_counts(normalize=True) * 100
hv.Bars(country_cnt).opts(title="Country Count", color="orange", xlabel="Countries", ylabel="Percentage", yformatter='%d%%')\
                .opts(opts.Bars(width=500, height=300,tools=['hover'],show_grid=True))\
            * hv.Text('Country_01', int(country_cnt.loc['Country_01']) + 2, f"{int(country_cnt.loc['Country_01'])}%")\
            * hv.Text('Country_02', int(country_cnt.loc['Country_02']) + 2, f"{int(country_cnt.loc['Country_02'])}%")\
            * hv.Text('Country_03', int(country_cnt.loc['Country_03']) + 2, f"{int(country_cnt.loc['Country_03'])}%")

In [ ]:
## 有修改！！
local_cnt = np.round(df['Local'].value_counts(normalize=True) * 100)
hv.Bars(local_cnt).opts(title="Local Count", color="orange", xlabel="Locals", ylabel="Percentage", yformatter='%d%%')\
                .opts(opts.Bars(width=700, height=300,tools=['hover'],show_grid=True))* hv.Curve(local_cnt).opts(color='red', line_width=3)\
                * hv.Text('Local_01', int(local_cnt.loc['Local_01']) + 1, f"{int(local_cnt.loc['Local_01'])}%", fontsize = 10)\
                * hv.Text('Local_02', int(local_cnt.loc['Local_02']) + 1, f"{int(local_cnt.loc['Local_02'])}%", fontsize = 10)\
                * hv.Text('Local_03', int(local_cnt.loc['Local_03']) + 1, f"{int(local_cnt.loc['Local_03'])}%", fontsize = 10)\
                * hv.Text('Local_04', int(local_cnt.loc['Local_04']) + 1, f"{int(local_cnt.loc['Local_04'])}%", fontsize = 10)\
                * hv.Text('Local_05', int(local_cnt.loc['Local_05']) + 1, f"{int(local_cnt.loc['Local_05'])}%", fontsize = 10)\
                * hv.Text('Local_06', int(local_cnt.loc['Local_06']) + 1, f"{int(local_cnt.loc['Local_06'])}%", fontsize = 10)\
                * hv.Text('Local_07', int(local_cnt.loc['Local_07']) + 1, f"{int(local_cnt.loc['Local_07'])}%", fontsize = 10)\
                * hv.Text('Local_08', int(local_cnt.loc['Local_08']) + 1, f"{int(local_cnt.loc['Local_08'])}%", fontsize = 10)\
                * hv.Text('Local_09', int(local_cnt.loc['Local_09']) + 1, f"{int(local_cnt.loc['Local_09'])}%", fontsize = 10)\
                * hv.Text('Local_10', int(local_cnt.loc['Local_10']) + 1, f"{int(local_cnt.loc['Local_10'])}%", fontsize = 10)\
                * hv.Text('Local_11', int(local_cnt.loc['Local_11']) + 1, f"{int(local_cnt.loc['Local_11'])}%", fontsize = 10)\
                * hv.Text('Local_12', int(local_cnt.loc['Local_12']) + 1, f"{int(local_cnt.loc['Local_12'])}%", fontsize = 10)    

In [ ]:
## 有修改！！
sector_cnt = np.round(df['Industry Sector'].value_counts(normalize=True) * 100)
hv.Bars(sector_cnt).opts(title="Industry Sector Count", color="orange", xlabel="Sectors", ylabel="Percentage", yformatter='%d%%')\
                .opts(opts.Bars(width=500, height=300,tools=['hover'],show_grid=True))\
                * hv.Text('Mining', int(sector_cnt.loc['Mining']) + 2, f"{int(sector_cnt.loc['Mining'])}%")\
                * hv.Text('Metals', int(sector_cnt.loc['Metals']) + 2, f"{int(sector_cnt.loc['Metals'])}%")\
                * hv.Text('Others', int(sector_cnt.loc['Others']) + 2, f"{int(sector_cnt.loc['Others'])}%")


In [ ]:
ac_level_cnt = np.round(df['Accident Level'].value_counts(normalize=True) * 100)
pot_ac_level_cnt = np.round(df['Potential Accident Level'].value_counts(normalize=True) * 100)

ac_pot = pd.concat([ac_level_cnt, pot_ac_level_cnt], axis=1,sort=False).fillna(0).rename(columns={'Accident Level':'Accident', 'Potential Accident Level':'Potential'}) # 两变量横向表对接
ac_pot = pd.melt(ac_pot.reset_index(), ['index']).rename(columns={'index':'Severity', 'variable':'Levels'}) 
hv.Bars(ac_pot, kdims = ['Severity', 'Levels'], vdims = ['value']).opts(title="Accident Levels Count", width=800, height=400,tools=['hover'], show_grid=True,xrotation=45, ylabel="Percentage", yformatter='%d%%')


In [ ]:
## 有改动！！
gender_cnt = np.round(df['Gender'].value_counts(normalize=True) * 100)
print(gender_cnt)
print(type(gender_cnt))
hv.Bars(gender_cnt).opts(title="Gender Count", color="orange", xlabel="Gender", ylabel="Percentage", yformatter='%d%%')\
                .opts(opts.Bars(width=300, height=300,tools=['hover'],show_grid=True))\
                    * hv.Text('Male', int(gender_cnt.loc['Male']) + 3, f"{int(gender_cnt.loc['Male'])}%")\
                    * hv.Text('Female', int(gender_cnt.loc['Female']) + 3, f"{int(gender_cnt.loc['Female'])}%")

In [ ]:
## 有改动！！
emp_type_cnt = np.round(df['Employee type'].value_counts(normalize=True) * 100)
hv.Bars(emp_type_cnt).opts(title="Employee type Count", color="orange", xlabel="Employee Type", ylabel="Percentage", yformatter='%d%%')\
                .opts(opts.Bars(width=500, height=300,tools=['hover'],show_grid=True))* hv.Curve(emp_type_cnt).opts(color='red', line_width=3)\
                    * hv.Text('Third Party', int(emp_type_cnt.loc['Third Party']) + 1.5, f"{int(emp_type_cnt.loc['Third Party'])}%", fontsize = 10)\
                    * hv.Text('Employee', int(emp_type_cnt.loc['Employee']) + 1.5, f"{int(emp_type_cnt.loc['Employee'])}%", fontsize = 10)\
                    * hv.Text('Third Party (Remote)', int(emp_type_cnt.loc['Third Party (Remote)']) + 1.5, f"{int(emp_type_cnt.loc['Third Party (Remote)'])}%", fontsize = 10)    


In [ ]:
## 有改动！！

## 占比小于1%的数据未进行标注
cr_risk_cnt = np.round(df['Critical Risk'].value_counts(normalize=True) * 100)
hv.Bars(cr_risk_cnt[::-1]).opts(title="Critical Risk Count", color="orange", xlabel="Critical Risks", ylabel="Percentage", xformatter='%d%%')\
                .opts(opts.Bars(width=800, height=800,tools=['hover'],show_grid=True,invert_axes=True))* hv.Curve(cr_risk_cnt).opts(color='red', line_width=3)\
                    * hv.Text('Others', int(cr_risk_cnt.loc['Others']) + 2, f"{int(cr_risk_cnt.loc['Others'])}%", fontsize = 9)\
                    * hv.Text('Pressed', int(cr_risk_cnt.loc['Pressed']) + 2, f"{int(cr_risk_cnt.loc['Pressed'])}%", fontsize = 9)\
                    * hv.Text('Manual Tools', int(cr_risk_cnt.loc['Manual Tools']) + 2, f"{int(cr_risk_cnt.loc['Manual Tools'])}%", fontsize = 9)\
                    * hv.Text('Chemical substances', int(cr_risk_cnt.loc['Chemical substances']) + 2, f"{int(cr_risk_cnt.loc['Chemical substances'])}%", fontsize = 9)\
                    * hv.Text('Venomous Animals', int(cr_risk_cnt.loc['Venomous Animals']) + 2, f"{int(cr_risk_cnt.loc['Venomous Animals'])}%", fontsize = 9)\
                    * hv.Text('Cut', int(cr_risk_cnt.loc['Cut']) + 2, f"{int(cr_risk_cnt.loc['Cut'])}%", fontsize = 9)\
                    * hv.Text('Projection', int(cr_risk_cnt.loc['Projection']) + 2, f"{int(cr_risk_cnt.loc['Projection'])}%", fontsize = 9)\
                    * hv.Text('Bees', int(cr_risk_cnt.loc['Bees']) + 2, f"{int(cr_risk_cnt.loc['Bees'])}%", fontsize = 9)\
                    * hv.Text('Fall', int(cr_risk_cnt.loc['Fall']) + 2, f"{int(cr_risk_cnt.loc['Fall'])}%", fontsize = 9)\
                    * hv.Text('Vehicles and Mobile Equipment', int(cr_risk_cnt.loc['Vehicles and Mobile Equipment']) + 2, f"{int(cr_risk_cnt.loc['Vehicles and Mobile Equipment'])}%", fontsize = 9)\
                    * hv.Text('Fall prevention (same level)', int(cr_risk_cnt.loc['Fall prevention (same level)']) + 2, f"{int(cr_risk_cnt.loc['Fall prevention (same level)'])}%", fontsize = 9)\
                    * hv.Text('remains of choco', int(cr_risk_cnt.loc['remains of choco']) + 2, f"{int(cr_risk_cnt.loc['remains of choco'])}%", fontsize = 9)\
                    * hv.Text('Pressurized Systems', int(cr_risk_cnt.loc['Pressurized Systems']) + 2, f"{int(cr_risk_cnt.loc['Pressurized Systems'])}%", fontsize = 9)\
                    * hv.Text('Fall prevention', int(cr_risk_cnt.loc['Fall prevention']) + 2, f"{int(cr_risk_cnt.loc['Fall prevention'])}%", fontsize = 9)\
                    * hv.Text('Suspended Loads', int(cr_risk_cnt.loc['Suspended Loads']) + 2, f"{int(cr_risk_cnt.loc['Suspended Loads'])}%", fontsize = 9)\
                    * hv.Text('Blocking and isolation of energies', int(cr_risk_cnt.loc['Blocking and isolation of energies']) + 2, f"{int(cr_risk_cnt.loc['Blocking and isolation of energies'])}%", fontsize = 9)\
                    * hv.Text('Pressurized Systems / Chemical Substances', int(cr_risk_cnt.loc['Pressurized Systems / Chemical Substances']) + 2, f"{int(cr_risk_cnt.loc['Pressurized Systems / Chemical Substances'])}%", fontsize = 9)\
                    * hv.Text('Power lock', int(cr_risk_cnt.loc['Power lock']) + 2, f"{int(cr_risk_cnt.loc['Power lock'])}%", fontsize = 9)\
                    * hv.Text('Liquid Metal', int(cr_risk_cnt.loc['Liquid Metal']) + 2, f"{int(cr_risk_cnt.loc['Liquid Metal'])}%", fontsize = 9)
                            

In [ ]:
year_cnt = np.round(df['Year'].value_counts(normalize=True,sort=False) * 100)
y = hv.Bars(year_cnt).opts(title="Year Count", color="orange", xlabel="Years")* hv.Curve(year_cnt).opts(color='red', line_width=3)
month_cnt = np.round(df['Month'].value_counts(normalize=True,sort=False) * 100)
m = hv.Bars(month_cnt).opts(title="Month Count", color="orange", xlabel="Months") * hv.Curve(month_cnt).opts(color='red', line_width=3)
day_cnt = np.round(df['Day'].value_counts(normalize=True,sort=False) * 100)
d = hv.Bars(day_cnt).opts(title="Day Count", color="orange", xlabel="Days") * hv.Curve(day_cnt).opts(color='red', line_width=3)
weekday_cnt = pd.DataFrame(np.round(df['Weekday'].value_counts(normalize=True,sort=False) * 100))
weekday_cnt['week_num'] = [['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'].index(i) for i in weekday_cnt.index]
weekday_cnt.sort_values('week_num', inplace=True)
w = hv.Bars((weekday_cnt.index, weekday_cnt.Weekday)).opts(title="Weekday Count", color="orange", xlabel="Weekdays") * hv.Curve(weekday_cnt['Weekday']).opts(color='red', line_width=3)
#(y + m + d + w).opts(opts.Bars(width=400, height=300,tools=['hover'],show_grid=True, ylabel="Percentage", yformatter='%d%%')).cols(2)
y.opts(opts.Bars(width=300, height=300,tools=['hover'],show_grid=True, ylabel="Percentage", yformatter='%d%%'))

In [ ]:
m.opts(opts.Bars(width=400, height=300,tools=['hover'],show_grid=True, ylabel="Percentage", yformatter='%d%%'))

In [ ]:
d.opts(opts.Bars(width=600, height=400,tools=['hover'],show_grid=True, ylabel="Percentage", yformatter='%d%%'))

In [ ]:
w.opts(opts.Bars(width=500, height=300,tools=['hover'],show_grid=True, ylabel="Percentage", yformatter='%d%%'))

In [ ]:
## 有改动！！
season_cnt = pd.DataFrame(np.round(df['Season'].value_counts(normalize=True,sort=False) * 100).reset_index())
season_cnt['season_order'] = season_cnt['index'].apply(lambda x: ['Spring','Summer','Autumn','Winter'].index(x))
season_cnt.sort_values('season_order', inplace=True)
season_cnt.index = season_cnt['index']
season_cnt.drop(['index','season_order'], axis=1, inplace=True)
#print(season_cnt)
hv.Bars(season_cnt).opts(title="Season Count", color="orange", xlabel="Season", ylabel="Percentage", yformatter='%d%%')\
                .opts(opts.Bars(width=400, height=300,tools=['hover'],show_grid=True)) * hv.Curve(season_cnt).opts(color='red', line_width=3)\
                    * hv.Text('Spring', int(season_cnt.loc['Spring']) + 1, f"{int(season_cnt.loc['Spring'])}%", fontsize = 10)\
                    * hv.Text('Summer', int(season_cnt.loc['Summer']) + 1, f"{int(season_cnt.loc['Summer'])}%", fontsize = 10)\
                    * hv.Text('Autumn', int(season_cnt.loc['Autumn']) + 1, f"{int(season_cnt.loc['Autumn'])}%", fontsize = 10)\
                    * hv.Text('Winter', int(season_cnt.loc['Winter']) + 1, f"{int(season_cnt.loc['Winter'])}%", fontsize = 10)

In [ ]:
f = lambda x : np.round(x/x.sum() * 100)
con_sector = df.groupby(['Country','Industry Sector'])['Industry Sector'].count().unstack().apply(f, axis=1)
hv.Bars(pd.melt(con_sector.reset_index(), ['Country']), ['Country', 'Industry Sector'], 'value').opts(opts.Bars(title="Industry Sector by Countries Count", width=800, height=300,tools=['hover'],\
                                                                show_grid=True,xrotation=0, ylabel="Percentage", yformatter='%d%%'))

In [ ]:
f = lambda x : np.round(x/x.sum() * 100)
em_gen = df.groupby(['Gender','Employee type'])['Employee type'].count().unstack().apply(f, axis=1)
hv.Bars(pd.melt(em_gen.reset_index(), ['Gender']), ['Gender','Employee type'], 'value').opts(opts.Bars(title="Employee type by Gender Count", width=800, height=300,tools=['hover'],\
                                                                show_grid=True,xrotation=0, ylabel="Percentage", yformatter='%d%%'))

In [ ]:

f = lambda x : np.round(x/x.sum() * 100)
em_gen = df.groupby(['Gender','Industry Sector'])['Industry Sector'].count().unstack().apply(f, axis=1)
hv.Bars(pd.melt(em_gen.reset_index(), ['Gender']), ['Gender','Industry Sector'], 'value').opts(opts.Bars(title="Industry Sector by Gender Count", width=800, height=300,tools=['hover'],\
                                                                show_grid=True,xrotation=0, ylabel="Percentage", yformatter='%d%%'))
                                                                    #* hv.Text(['Female', 'Metals'], int(em_gen.loc['Female']['Metals']) + 1, f"{int(em_gen.loc['Female']['Metals'])}%", fontsize = 10)

In [ ]:
f = lambda x : np.round(x/x.sum() * 100)
ac_gen = df.groupby(['Gender','Accident Level'])['Accident Level'].count().unstack().apply(f, axis=1)
ac = hv.Bars(pd.melt(ac_gen.reset_index(), ['Gender']), ['Gender','Accident Level'], 'value').opts(opts.Bars(title="Accident Level by Gender Count"))
pot_ac_gen = df.groupby(['Gender','Potential Accident Level'])['Potential Accident Level'].count().unstack().apply(f, axis=1)
pot_ac = hv.Bars(pd.melt(pot_ac_gen.reset_index(), ['Gender']), ['Gender','Potential Accident Level'], 'value').opts(opts.Bars(title="Potential Accident Level by Gender Count"))
(ac + pot_ac).opts(opts.Bars(width=400, height=300,tools=['hover'],show_grid=True,xrotation=0, ylabel="Percentage", yformatter='%d%%'))

In [ ]:
f = lambda x : np.round(x/x.sum() * 100)
ac_em = df.groupby(['Employee type','Accident Level'])['Accident Level'].count().unstack().apply(f, axis=1)
ac = hv.Bars(pd.melt(ac_em.reset_index(), ['Employee type']), ['Employee type','Accident Level'], 'value').opts(opts.Bars(title="Accident Level by Employee type Count"))
pot_ac_em = df.groupby(['Employee type','Potential Accident Level'])['Potential Accident Level'].count().unstack().apply(f, axis=1)
pot_ac = hv.Bars(pd.melt(pot_ac_em.reset_index(), ['Employee type']), ['Employee type','Potential Accident Level'], 'value').opts(opts.Bars(title="Potential Accident Level by Employee type Count"))
(ac + pot_ac).opts(opts.Bars(width=400, height=300,tools=['hover'],show_grid=True,xrotation=0, ylabel="Percentage", yformatter='%d%%',fontsize={'title':9}))

In [ ]:
f = lambda x : np.round(x/x.sum() * 100)
ac_mo = df.groupby(['Month','Accident Level'])['Accident Level'].count().unstack().apply(f, axis=1).fillna(0)
ac = hv.Curve(ac_mo['I'], label='I') * hv.Curve(ac_mo['II'], label='II') * hv.Curve(ac_mo['III'], label='III') * hv.Curve(ac_mo['IV'], label='IV') * hv.Curve(ac_mo['V'], label='V')\
        .opts(opts.Curve(title="Accident Level by Month Count"))
pot_ac_mo = df.groupby(['Month','Potential Accident Level'])['Potential Accident Level'].count().unstack().apply(f, axis=1).fillna(0)
pot_ac = hv.Curve(pot_ac_mo['I'], label='I') * hv.Curve(pot_ac_mo['II'], label='II') * hv.Curve(pot_ac_mo['III'], label='III') * hv.Curve(pot_ac_mo['IV'], label='IV')\
        * hv.Curve(pot_ac_mo['V'], label='V') * hv.Curve(pot_ac_mo['VI'], label='VI').opts(opts.Curve(title="Potential Accident Level by Month Count"))
(ac+pot_ac).opts(opts.Curve(width=800, height=300,tools=['hover'],show_grid=True, ylabel="Percentage", yformatter='%d%%')).cols(1)

In [ ]:
f = lambda x : np.round(x/x.sum() * 100)
ac_weekday = df.groupby(['Weekday','Accident Level'])['Accident Level'].count().unstack().apply(f, axis=1).fillna(0)
ac_weekday['week_num'] = [['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'].index(i) for i in ac_weekday.index]
ac_weekday.sort_values('week_num', inplace=True)
ac_weekday.drop('week_num', axis=1, inplace=True)
ac = hv.Curve(ac_weekday['I'], label='I') * hv.Curve(ac_weekday['II'], label='II') * hv.Curve(ac_weekday['III'], label='III') * hv.Curve(ac_weekday['IV'], label='IV') * hv.Curve(ac_weekday['V'], label='V')\
        .opts(opts.Curve(title="Accident Level by Weekday Count"))
pot_ac_weekday = df.groupby(['Weekday','Potential Accident Level'])['Potential Accident Level'].count().unstack().apply(f, axis=0).fillna(0)
pot_ac_weekday['week_num'] = [['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'].index(i) for i in pot_ac_weekday.index]
pot_ac_weekday.sort_values('week_num', inplace=True)
pot_ac_weekday.drop('week_num', axis=1, inplace=True)
pot_ac = hv.Curve(pot_ac_weekday['I'], label='I') * hv.Curve(pot_ac_weekday['II'], label='II') * hv.Curve(pot_ac_weekday['III'], label='III') * hv.Curve(pot_ac_weekday['IV'], label='IV')\
        * hv.Curve(pot_ac_weekday['V'], label='V') * hv.Curve(pot_ac_weekday['VI'], label='VI').opts(opts.Curve(title="Potential Accident Level by Weekday Count"))
(ac+pot_ac).opts(opts.Curve(width=800, height=300,tools=['hover'],show_grid=True, ylabel="Percentage", yformatter='%d%%')).cols(1)

In [ ]:
f = lambda x : np.round(x/x.sum() * 100)
ac_season = df.groupby(['Season','Accident Level'])['Accident Level'].count().unstack().apply(f, axis=1).fillna(0)
ac_season['season_num'] = [['Spring', 'Summer', 'Autumn', 'Winter'].index(i) for i in ac_season.index]
ac_season.sort_values('season_num', inplace=True)
ac_season.drop('season_num', axis=1, inplace=True)
ac = hv.Curve(ac_season['I'], label='I') * hv.Curve(ac_season['II'], label='II') * hv.Curve(ac_season['III'], label='III') * hv.Curve(ac_season['IV'], label='IV') * hv.Curve(ac_season['V'], label='V')\
        .opts(opts.Curve(title="Accident Level by Season Count"))
pot_ac_season = df.groupby(['Season','Potential Accident Level'])['Potential Accident Level'].count().unstack().apply(f, axis=0).fillna(0)
pot_ac_season['season_num'] = [['Spring', 'Summer', 'Autumn', 'Winter'].index(i) for i in pot_ac_season.index]
pot_ac_season.sort_values('season_num', inplace=True)
pot_ac_season.drop('season_num', axis=1, inplace=True)
pot_ac = hv.Curve(pot_ac_season['I'], label='I') * hv.Curve(pot_ac_season['II'], label='II') * hv.Curve(pot_ac_season['III'], label='III') * hv.Curve(pot_ac_season['IV'], label='IV')\
        * hv.Curve(pot_ac_season['V'], label='V') * hv.Curve(pot_ac_season['VI'], label='VI').opts(opts.Curve(title="Potential Accident Level by Season Count"))
(ac+pot_ac).opts(opts.Curve(width=800, height=300,tools=['hover'],show_grid=True, ylabel="Percentage", yformatter='%d%%')).cols(1)

# 5. 特征工程

In [ ]:
# 有修改
feature_df = pd.DataFrame()
for i in [1, 2]:
    vec_tfidf = TfidfVectorizer(max_features=30, norm='l2', stop_words='english', lowercase=True, use_idf=True, ngram_range=(i,i))
    X = vec_tfidf.fit_transform(df['Description_processed']).toarray()
    tfs = pd.DataFrame(X, columns=["TFIDF_" + n for n in vec_tfidf.get_feature_names_out()])
    feature_df = pd.concat([feature_df, tfs], axis=1)
feature_df = pd.concat([df, feature_df], axis=1)
feature_df.head(5)

In [ ]:
# 类别归类
feature_df["Accident_left hand"] = (feature_df["TFIDF_finger left"] + feature_df["TFIDF_left hand"]).apply(lambda x: 1 if x > 0 else 0)
feature_df["Accident_right hand"] = (feature_df["TFIDF_right hand"] + feature_df["TFIDF_finger right"]).apply(lambda x: 1 if x > 0 else 0)
feature_df["Accident_right leg"] = (feature_df["TFIDF_right leg"]).apply(lambda x: 1 if x > 0 else 0)
feature_df["Accident_left leg"] = (feature_df["TFIDF_left foot"]).apply(lambda x: 1 if x > 0 else 0)
feature_df["Accident_being cut"] = (feature_df["TFIDF_cut"] + feature_df["TFIDF_caus cut"]).apply(lambda x: 1 if x > 0 else 0)
feature_df["Accident_being hit"] = (feature_df["TFIDF_hit"] + feature_df["TFIDF_hit right"]).apply(lambda x: 1 if x > 0 else 0)
feature_df["Accident_falling"] = (feature_df["TFIDF_fall"]).apply(lambda x: 1 if x > 0 else 0)
# feature_df["Accident_fragment rock"] = (feature_df["TFIDF_fragment rock"]).apply(lambda x: 1 if x > 0 else 0)

In [ ]:
#删去原来TFIDF提取出的特征
feature_df = feature_df[feature_df.columns.drop(list(feature_df.filter(regex='TFIDF')))]
feature_df.columns

In [ ]:
feature_df['Country'] = LabelEncoder().fit_transform(feature_df['Country']).astype(np.int8)
feature_df['Local'] = LabelEncoder().fit_transform(feature_df['Local']).astype(np.int8)
feature_df['Industry Sector'] = LabelEncoder().fit_transform(feature_df['Industry Sector']).astype(np.int8)
feature_df['Accident Level'] = LabelEncoder().fit_transform(feature_df['Accident Level']).astype(np.int8)
feature_df['Potential Accident Level'] = LabelEncoder().fit_transform(feature_df['Potential Accident Level']).astype(np.int8)
feature_df['Gender'] = LabelEncoder().fit_transform(feature_df['Gender']).astype(np.int8)
feature_df['Employee type'] = LabelEncoder().fit_transform(feature_df['Employee type']).astype(np.int8)
feature_df['Critical Risk'] = LabelEncoder().fit_transform(feature_df['Critical Risk']).astype(np.int8)
feature_df['Weekday'] = LabelEncoder().fit_transform(feature_df['Weekday']).astype(np.int8)
feature_df['Season'] = LabelEncoder().fit_transform(feature_df['Season']).astype(np.int8)
feature_df.drop(['Date','Description', 'Description_processed'],axis=1,inplace=True)
feature_df.head(3)

# 6. 损失分析

In [ ]:
ac_level_cnt = df['Accident Level'].value_counts(normalize=True) * 100
pot_ac_level_cnt = df['Potential Accident Level'].value_counts(normalize=True) * 100
ac_pot = pd.concat([ac_level_cnt, pot_ac_level_cnt], axis=1,sort=False).fillna(0).rename(columns={'Accident Level':'Accident', 'Potential Accident Level':'Potential'})
ac_pot = pd.melt(ac_pot.reset_index(), ['index']).rename(columns={'index':'Severity', 'variable':'Levels'})
hv.Bars(ac_pot, ['Severity', 'Levels'], 'value').opts(opts.Bars(title="Accident Levels Count", width=700, height=300,tools=['hover'],\
                                                                      show_grid=True,xrotation=45, ylabel="Percentage", yformatter='%d%%'))           

In [ ]:
df.insert(5, 'Ac_Loss', 0)
df.insert(7, 'Pot_Ac_Loss', 0)
ac_level_P = ac_level_cnt / 100
pot_ac_level_P = ac_level_P

In [ ]:
lambda_1 = 1
lambda_2 = 0.8

## Accident Loss
df.loc[df['Accident Level']=='I', 'Ac_Loss'] = lambda_1 * 1 * ac_level_P[0]
df.loc[df['Accident Level']=='II', 'Ac_Loss'] = lambda_1 * 2 * ac_level_P[1]
df.loc[df['Accident Level']=='III', 'Ac_Loss'] = lambda_1 * 3 * ac_level_P[2]
df.loc[df['Accident Level']=='IV', 'Ac_Loss'] = lambda_1 * 4 * ac_level_P[3]
df.loc[df['Accident Level']=='V', 'Ac_Loss'] = lambda_1 * 5 * ac_level_P[4]

# df.loc[df['Accident Level']=='I', 'Ac_Loss'] = lambda_1 * 1 
# df.loc[df['Accident Level']=='II', 'Ac_Loss'] = lambda_1 * 2
# df.loc[df['Accident Level']=='III', 'Ac_Loss'] = lambda_1 * 3
# df.loc[df['Accident Level']=='IV', 'Ac_Loss'] = lambda_1 * 4 
# df.loc[df['Accident Level']=='V', 'Ac_Loss'] = lambda_1 * 5
## Potential Accident Loss 
df.loc[df['Potential Accident Level']=='I', 'Pot_Ac_Loss'] = lambda_2 * 1 * pot_ac_level_P[0]
df.loc[df['Potential Accident Level']=='II', 'Pot_Ac_Loss'] = lambda_2 * 2 * pot_ac_level_P[1]
df.loc[df['Potential Accident Level']=='III', 'Pot_Ac_Loss'] = lambda_2 * 3 * pot_ac_level_P[2]
df.loc[df['Potential Accident Level']=='IV', 'Pot_Ac_Loss'] = lambda_2 * 4 * pot_ac_level_P[3]
df.loc[df['Potential Accident Level']=='V', 'Pot_Ac_Loss'] = lambda_2 * 5 * pot_ac_level_P[4]

In [ ]:
df.insert(8, 'Total_Ac_Loss', 0)
df['Total_Ac_Loss'] = df['Ac_Loss'] + df['Pot_Ac_Loss']
df['Total_Ac_Loss'] = df['Total_Ac_Loss'].round(2)

In [ ]:
category = [0, 1, 2, 3, 4]
feature_df['Total_Ac_Loss']=pd.cut(feature_df['Total_Ac_Loss'], 5, labels=category)
feature_df['Total_Ac_Loss']

In [ ]:
y_series = feature_df['Total_Ac_Loss']
y_series = y_series.astype('int64')
# y_series = feature_df['Accident Level']
accident_part = ['Accident_left leg','Accident_right leg', 'Accident_left hand', 'Accident_right hand']
accident_type = ['Accident_falling','Accident_being cut', 'Accident_being hit']
x_df = feature_df.drop(['Potential Accident Level','Accident Level', 'Total_Ac_Loss', 'Ac_Loss', 'Pot_Ac_Loss'], axis=1) 
part_category = feature_df[accident_part]
type_category = feature_df[accident_type]
objective = x_df.drop(accident_type, axis=1)
objective = objective.drop('Description_sentiment_score', axis=1)
objective = objective.drop(accident_part, axis=1)
# X_train, X_valid, Y_train, Y_valid = train_test_split(x_df, y_series, test_size=0.2, random_state=0, stratify=y_series)
X1_train, X1_valid, Y1_train, Y1_valid = train_test_split(part_category, y_series, test_size=0.2, random_state=0, stratify=y_series)
X2_train, X2_valid, Y2_train, Y2_valid = train_test_split(type_category, y_series, test_size=0.2, random_state=0, stratify=y_series)
X3_train, X3_valid, Y3_train, Y3_valid = train_test_split(objective, y_series, test_size=0.2, random_state=0, stratify=y_series)

# lgb_train = lgb.Dataset(X_train, Y_train)
lgb1_train = lgb.Dataset(X1_train, Y1_train)
lgb2_train = lgb.Dataset(X2_train, Y2_train)
lgb3_train = lgb.Dataset(X3_train, Y3_train)
lgb1_valid = lgb.Dataset(X1_valid, Y1_valid, reference=lgb1_train)
lgb2_valid = lgb.Dataset(X2_valid, Y2_valid, reference=lgb2_train)
lgb3_valid = lgb.Dataset(X3_valid, Y3_valid, reference=lgb3_train)

In [ ]:
params = {
    'task' : 'train',
    'boosting' : 'gbdt',
    'objective': 'multiclass',
    'num_class': 5,
    'metric': 'multi_logloss',
    'num_leaves': 200,
    'feature_fraction': 1.0,
    'bagging_fraction': 1.0,
    'bagging_freq': 0,
    'min_child_samples': 5
}
gbm_ac1 = lgb.train(params,
            lgb1_train,
            num_boost_round=100,
            valid_sets=lgb1_valid,
            early_stopping_rounds=100)
gbm_ac2 = lgb.train(params,
            lgb2_train,
            num_boost_round=100,
            valid_sets=lgb2_valid,
            early_stopping_rounds=100)
gbm_ac3 = lgb.train(params,
            lgb3_train,
            num_boost_round=100,
            valid_sets=lgb3_valid,
            early_stopping_rounds=100)

In [ ]:
ac_label = ['Loss Level : I','Loss Level : II','Loss Level : III','Loss Level : IV','Loss Level : V']
explainer = shap.TreeExplainer(model=gbm_ac1)
# explainer = shap.TreeExplainer(model=xgbboost1)
shap_values_ac = explainer.shap_values(X=X1_train)
shap.summary_plot(shap_values=shap_values_ac, features=X1_train, feature_names=X1_train.columns, plot_type="bar", max_display=30, class_names=ac_label)

In [ ]:
explainer = shap.TreeExplainer(model=gbm_ac2)
# explainer = shap.TreeExplainer(model=gbm_ac2)
shap_values_ac = explainer.shap_values(X=X2_train)
shap.summary_plot(shap_values=shap_values_ac, features=X2_train, feature_names=X2_train.columns, plot_type="bar", max_display=1000, class_names=ac_label)

In [ ]:
explainer = shap.TreeExplainer(model=gbm_ac3)
# explainer = shap.TreeExplainer(model=gbm_ac2)
shap_values_ac = explainer.shap_values(X=X3_train)
shap.summary_plot(shap_values=shap_values_ac, features=X3_train, feature_names=X3_train.columns, plot_type="bar", max_display=30, class_names=ac_label)

In [ ]:
t = lgb.plot_tree(gbm_ac3, figsize=(20, 20), precision=1, tree_index=0, show_info=['split_gain'])
plt.title('Dicision Tree of Objective Reasons')
plt.show()